In [1]:
#@title Default title text
# Data processing modules
import pandas as pd
import numpy as np

# Machine learning algorithms
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm

# Model saving
import pickle

# Data cleaning
import re
import itertools
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords_eng = stopwords.words('english')

import string

from unidecode import unidecode

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kenem001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df_original = pd.read_csv('../Data/data_cleaned.csv')

df = df_original
df = df[df['Description'].notna()]
df = df[df['Translated'].notna()]

In [9]:
messages = df[df['Rep_Class'] == 'Warranty/Statutory Commercial Guarantees']['Translated']
messages.count()

961

In [10]:
X = ""

for i in messages:
    X += ' ' + i

freq = []

for unique in set(X.split(' ')):
    freq.append([X.split(' ').count(unique), unique])


freq = sorted(freq)
freq.reverse()

for i in freq:
    print(f"'{i[1]}', ", end='')

'warranty', 'not', 'consumer', 'guarantee', 'faulty', 'asked', 'query', 'gurantee', 'trader', 'bought', 'name', 'would', 'honoured', 'year', 'repairs', 'given', 'rights', 'still', 'contentbr', 'info', 'years', 'repair', 'told', 'replacement', 'legal', 'since', 'laptop', 'car', 'client', 'covered', 'gr', 'new', 'one', 'time', '2', 'wants', 'purchased', 'company', 'phone', 'months', 'replaced', 'mobile', 'know', 'defective', 'fault', 'said', 'issues', 'back', 'took', 'commercial', 'asking', 'also', 'weeks', 'like', '3', 'working', 'called', 'month', 'ago', '5', '1', 'sent', 'problem', 'whether', 'supplier', 'issue', 'expired', 'purchase', 'item', 'informed', 'td', 'repaired', 'period', 'malta', 'however', 'fiat', 'could', 'thanks', 'please', 'pay', 'parts', 'two', 'part', 'give', 'contacted', 'another', 'use', 'take', 'regarding', 'last', 'get', 'contact', '2020', 'unit', 'good', 'email', 'ask', 'tv', 'service', 'rust', 'long', 'lift', 'entitled', 'due', 'check', 'us', 'times', 'technici

In [28]:
def clean(input_text):
    '''
    This function cleans the body received from e-mails so that features
    to be processed by the Support Vector Machine are extracted from it 
    more effectively

    via https://www.geeksforgeeks.org/python-efficient-text-data-cleaning/
    '''

    apos_dict = {"'s":" is","n't":" not","'m":" am","'ll":" will",
           "'d":" would","'ve":" have","'re":" are"}

    warning_message = 'CAUTION: This email originated from OUTSIDE the Government Email Infrastructure. DO NOT CLICK LINKS or OPEN attachments unless you recognise the sender and know the content is safe.'

    if warning_message in input_text:
        input_text = re.sub(warning_message, '', input_text)

    # Convert to unicode
    input_text = unidecode(input_text) 

    # Remove URLs, hashtags
    input_text = re.sub(r'https?:\/\/.\S+', '', input_text)
    input_text = re.sub(r'#', '', input_text)

    # Contraction replacement
    for key, value in apos_dict.items():
        if key in input_text:
            input_text = input_text.replace(key,value)

    # Split attached words
    input_text = ' '.join([s for s in re.split('([A-Z][a-z]+[^A-Z]*)', input_text) if s])

    # Remove punctuation
    input_text_list = []
    for word in input_text:
        if word not in string.punctuation:
            input_text_list.append(word)

    input_text_list = ''.join(input_text_list).split(' ')
    input_text = ' '.join(input_text_list)

    # Translation + Spell check
    input_text = re.sub(r'(?<![\w\d])kons(?![\w\d])', 'konsumatur', input_text)
    input_text = re.sub(r'(?<![\w\d])cons(?![\w\d])', 'consumer', input_text)

    # translator = Translator()

    # lang = translator.detect(input_text).lang

    # input_text = translator.translate(input_text, dest='en').text
    # input_text_list = input_text.split(' ')

    spell = Speller(lang='en')
    
    for i in range(len(input_text_list)):
        input_text_list[i] = spell(input_text_list[i])

    input_text = ' '.join(input_text_list)

    # Convert to lowercase
    input_text = input_text.lower()

    # Remove stopwords
    exceptions = ['not']
    ignore = ['hi', 'hello', 'dear', 'sir', 'madam', 'ms', 'mr', 'regards']

    input_text_tokens = input_text.split()

    input_text_list = []
    for word in input_text_tokens:
        if (word not in stopwords_eng) and (word not in ignore) or (word in exceptions):
            input_text_list.append(word)

    input_text = ' '.join(input_text_list)

    input_text = re.sub(' +', ' ', input_text)

    return input_text, lang